In [39]:
import nibabel as nii
import numpy as np
import os
import tqdm

from skimage import data
from skimage import transform as tf
import matplotlib.pyplot as plt

In [40]:
train_path = '/home/localadmin/segthor/data/train_resampled_cropped'
output_path = '/home/localadmin/segthor/data/train_resampled2_2_25'

In [41]:
file_list = [f for f in os.listdir(train_path) if os.path.isdir(os.path.join(train_path, f))]
file_list.sort()
print(file_list)

['Patient_01', 'Patient_02', 'Patient_03', 'Patient_04', 'Patient_05', 'Patient_06', 'Patient_07', 'Patient_08', 'Patient_09', 'Patient_10', 'Patient_11', 'Patient_12', 'Patient_13', 'Patient_14', 'Patient_15', 'Patient_16', 'Patient_17', 'Patient_18', 'Patient_19', 'Patient_20', 'Patient_21', 'Patient_22', 'Patient_23', 'Patient_24', 'Patient_25', 'Patient_26', 'Patient_27', 'Patient_28', 'Patient_29', 'Patient_30', 'Patient_31', 'Patient_32', 'Patient_33', 'Patient_34', 'Patient_35', 'Patient_36', 'Patient_37', 'Patient_38', 'Patient_39', 'Patient_40']


In [42]:
def read_nii(file_path):
    return nii.load(file_path)

In [43]:
sizes = np.zeros(shape=(3,len(file_list)),dtype=float)
shapes = np.zeros(shape=(3,len(file_list)),dtype=float)
scales = np.zeros(shape=(3,len(file_list)),dtype=float)

for i,f in enumerate(file_list):
    filename = os.path.join(train_path,f,f+'.nii.gz')
    image=read_nii(filename)
    shapes[:,i]=image.header['dim'][1:4]
    scales[:,i]=image.header['pixdim'][1:4]
    sizes[:,i]=image.header['dim'][1:4] * image.header['pixdim'][1:4]
    print(f + '\t', shapes[:,i],'\t',scales[:,i],'\t',sizes[:,i])

Patient_01	 [451. 281. 304.] 	 [1.  1.  1.5] 	 [451. 281. 456.]
Patient_02	 [481. 301. 409.] 	 [1.  1.  1.5] 	 [481.  301.  613.5]
Patient_03	 [490. 238. 244.] 	 [1.  1.  1.5] 	 [490. 238. 366.]
Patient_04	 [453. 270. 262.] 	 [1.  1.  1.5] 	 [453. 270. 393.]
Patient_05	 [475. 277. 378.] 	 [1.  1.  1.5] 	 [475. 277. 567.]
Patient_06	 [454. 282. 331.] 	 [1.  1.  1.5] 	 [454.  282.  496.5]
Patient_07	 [446. 272. 238.] 	 [1.  1.  1.5] 	 [446. 272. 357.]
Patient_08	 [471. 322. 284.] 	 [1.  1.  1.5] 	 [471. 322. 426.]
Patient_09	 [411. 252. 256.] 	 [1.  1.  1.5] 	 [411. 252. 384.]
Patient_10	 [490. 313. 216.] 	 [1.  1.  1.5] 	 [490. 313. 324.]
Patient_11	 [549. 362. 299.] 	 [1.  1.  1.5] 	 [549.  362.  448.5]
Patient_12	 [416. 270. 227.] 	 [1.  1.  1.5] 	 [416.  270.  340.5]
Patient_13	 [477. 282. 303.] 	 [1.  1.  1.5] 	 [477.  282.  454.5]
Patient_14	 [425. 268. 269.] 	 [1.  1.  1.5] 	 [425.  268.  403.5]
Patient_15	 [458. 268. 249.] 	 [1.  1.  1.5] 	 [458.  268.  373.5]
Patient_16	 [499. 3

In [44]:
print(np.max(sizes, axis =1))
print(np.max(shapes, axis =1))
print(np.min(shapes, axis =1))

[567.  498.  613.5]
[567. 498. 409.]
[408. 238. 216.]


In [7]:

from scipy import ndimage

def m(coords, new_scale, old_scale, translate):
    return coords* new_scale / old_scale + translate

def warp(data, new_scale, old_scale, translation, output_shape, cval, order):
    coordinates = np.zeros(shape=(3,)+output_shape,dtype=float)
    for i in range(output_shape[0]):
        for j in range(output_shape[1]):
            for k in range(output_shape[2]):
                coordinates[:,i,j,k]=m(np.array([i,j,k]), new_scale, old_scale, translation)
    return ndimage.map_coordinates(data, coordinates, order=order, cval=cval, mode='constant')

In [8]:
def closest_to_k(n,k=8):
    if n % k == 0:
        return n
    else:
        return ((n // k) + 1)*k

In [11]:
#new_shape = (512,512,512)
#new_size = np.array([180.0,180.0,160.0])
new_scale = np.array([2.0,2.0,2.5])#new_size / new_shape
#new_size = new_scale * new_shape

affine = np.diag(np.append(new_scale,1))

In [12]:
from scipy.ndimage import zoom

In [13]:
file_list.sort()
for i,f in enumerate(file_list):
    filename = os.path.join(train_path, f,f+'.nii.gz')
    label_name = os.path.join(train_path, f,'GT.nii.gz')
    output_filename = os.path.join(output_path, f,f+'.nii.gz')
    output_label_name = os.path.join(output_path, f,'GT.nii.gz')
        
    image_handle = read_nii(filename)
    label_handle = read_nii(label_name)
    image_data = image_handle.get_data().astype(np.float32)
    label_data = label_handle.get_data().astype(np.int32)
    label_oh = np.eye(5)[label_data].astype(np.float32)
        
    shape = image_handle.header['dim'][1:4]
    scale = image_handle.header['pixdim'][1:4]
    size = image_handle.header['dim'][1:4] * image_handle.header['pixdim'][1:4]
        
    scale_factor = scale / new_scale
        
    new_shape = np.ceil(shape * scale_factor).astype(np.int32)
    new_shape = tuple(closest_to_k(i,8) for i in new_shape)
    new_size = new_scale * new_shape
        
    translation = np.array([(size[0] - new_size[0]) / 2, (size[1] - new_size[1]) / 2, (size[2] - new_size[2])/2])
    translation /= scale
        
    order = 3
        
    new_data = zoom(image_data, scale_factor, order=3, mode='constant', cval=-1024)
    new_label = zoom(label_oh, tuple(scale_factor)+(1,), order=1, mode='constant', cval=0)
    #print(new_label.shape)
    #new_data = warp(image_data, new_scale, scale, translation, new_shape, image_data.min(), order)
    new_image = nii.Nifti1Image(new_data.astype(np.float32), affine)
    new_label = nii.Nifti1Image(np.argmax(new_label,axis=-1).astype(np.float32), affine)
    os.mkdir(os.path.join(output_path, f))
    nii.save(new_image,output_filename)
    nii.save(new_label,output_label_name)
        
    print(f, str(new_shape))

Patient_01 (256, 256, 184)


/home/localadmin/anaconda2/envs/py36/lib/python3.6/site-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Patient_02 (256, 256, 248)
Patient_03 (256, 256, 152)
Patient_04 (256, 256, 160)
Patient_05 (256, 256, 232)
Patient_06 (256, 256, 200)
Patient_07 (256, 256, 144)
Patient_08 (256, 256, 176)
Patient_09 (256, 256, 160)
Patient_10 (256, 256, 136)
Patient_11 (328, 328, 184)
Patient_12 (256, 256, 144)
Patient_13 (256, 256, 184)
Patient_14 (256, 256, 168)
Patient_15 (232, 232, 152)
Patient_16 (256, 256, 168)
Patient_17 (256, 256, 168)
Patient_18 (256, 256, 184)
Patient_19 (256, 256, 192)
Patient_20 (352, 352, 208)
Patient_21 (256, 256, 160)
Patient_22 (256, 256, 184)
Patient_23 (256, 256, 160)
Patient_24 (256, 256, 168)
Patient_25 (256, 256, 152)
Patient_26 (256, 256, 200)
Patient_27 (256, 256, 176)
Patient_28 (256, 256, 184)
Patient_29 (256, 256, 176)
Patient_30 (256, 256, 176)
Patient_31 (256, 256, 152)
Patient_32 (256, 256, 160)
Patient_33 (256, 256, 216)
Patient_34 (256, 256, 192)
Patient_35 (352, 352, 152)
Patient_36 (256, 256, 152)
Patient_37 (256, 256, 176)
Patient_38 (256, 256, 160)
P